In [78]:
import openai

client = openai.OpenAI()
messages = []



In [79]:
def get_weather(city):
    return "33 degress celcius."


FUNCTION_MAP = {"get_weather": get_weather}

In [80]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of."
                    }
                }
            },
            "required": ["city"],
        }
    }
]

In [81]:
from openai.types.chat import ChatCompletionMessage


def process_ai_response(ai_message: ChatCompletionMessage):
    if ai_message.tool_calls:
        messages.append({
            "role": "assistant",
            "content": ai_message.content or "",
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments,
                    }
                }
                for tool_call in ai_message.tool_calls
            ]
        })

        for tool_call in ai_message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"🔥 Calling {function_name} with {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP[function_name]
            result = function_to_run(**arguments)
            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result
                }
            )
            call_ai()
    else:
        messages.append({
            "role": "assistant",
            "content": ai_message.content,
        })
        print(f"🔵 AI: {ai_message.content}")


In [82]:
import json


def call_ai():
    response = client.chat.completions.create(
        model="gpt-5-nano-2025-08-07",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)


In [ ]:
while True:
    message = input("Send the message to LLM")
    if message == 'quit' or message == 'q':
        break
    else:
        messages.append({
            "role": "user",
            "content": message
        })
    print(f"🟢 User: {message}")
    call_ai()